In [ ]:
#This Program predicts Stock prices

In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Get the Stock Data
df = web.DataReader('ICICIBANK.BO',data_source = 'yahoo',start='2019-06-03',end='2021-05-10')
#Print Data
df

In [ ]:
df.shape

In [ ]:
#Visualize the closing price
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price (₹)',fontsize=18)
plt.show()

In [ ]:
#CREATE a new dataframe with only close column
data = df.filter(['Close'])
#Convert the dataframe to numpy array
dataset = data.values
#Get the number of arrays to train the model on
training_data_len = math.ceil(len(dataset)*0.8)

training_data_len

In [ ]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
#Create the training dataset
#Create the scaled training dataset
train_data = scaled_data[0:training_data_len,:]
#Split the data into x_train and y_train
x_train = []
y_train = []

for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if i<=61:
        print(x_train)
        print(y_train)
        print()

In [ ]:
#Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshape the Data
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
#build the LSTM Model
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
#Train the model
model.fit(x_train,y_train,batch_size=1,epochs=1)

In [ ]:
training_data_len

In [ ]:
#Create the testing dataset
#Create new array containing skill values from index 2239 to 2798
test_data = scaled_data[training_data_len-60:, :]

In [ ]:
test_data

In [ ]:
#Create the data_sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
 #Convert the data to a numpy array
x_test = np.array(x_test)

In [ ]:
#Reshape the data
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)


In [ ]:
#RMSE
rmse = np.sqrt(np.mean(predictions-y_test)**2)
rmse

In [ ]:
#plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Visualize DAta
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price ₹',fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'], loc='lower right')
plt.show()

In [ ]:
#Show the Valid Actual and Predicted Price
valid

In [ ]:
#Get the quote
icici_quote = web.DataReader('ICICIBANK.BO', data_source='yahoo', start='2019-06-03', end='2021-05-19')
#New Data Frame
new_df= icici_quote.filter(['Close'])
#Get the last 60 closing value
last_60_days = new_df[-60:].values
#Scale the data
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
#Append past 60 days to X_test
X_test.append(last_60_days_scaled)
#Convert X_test to a nmpy array
X_test = np.array(X_test)
#Reshape the data to be 3D
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
icici_quote2 = web.DataReader('ICICIBANK.BO', data_source='yahoo', start='2019-06-03', end='2021-05-19')
icici_quote2.tail(2)